[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//IIS-KI-Tutorials/KI-Tutorials/blob/main/active-learning/4_Interaktiver_Active_Learning_Cycle_mit_Superintendent.ipynb)

# Interactives Labeling mit Superintendent
Bisher sind wir in unseren Beispielen immer davon ausgegangen, dass man eine Art Orakel hat, welches einem das Label für einen Datenpunkt verrät. Meist existiert ein solches Orakel jedoch nicht und man muss selber als Mensch das richtige Label bestimmen.  Wie dieser "human -in-the-loop" Ansatz konkret aussehen und effizient umgesetzt werden kann, zeigt dieses Beispiel. Mit Hilfe der Bibliothek Superintendent lässt sich mit nur wenig Code ein interactives Widget zur Umsetzung des Activ Learning Cylces erzeugen. Als Beispielaufgabe wählen wir die Klassifikation der Ziffern von 0, 1, 2 und 3 auf dem MNIST Datensatz. Ziel ist es, die jeweilige handgeschriebene Ziffer korrekt zu erkennen.

## Inhaltsverzeichnis
1. [Imports](#Imports)
2. [Laden der Daten](#Laden-der-Daten)
3. [Initialisiere Widget](#Initialisiere-Widget)
4. [Integriere Widget in Active Learning Cycle](#Integriere-Widget-in-Active-Learning-Cycle)

### 1. Imports <a id='Imports'></a>
Neben den gängigen Python-Bibliotheken wie `numpy` und `pandas` sind in diesem Code insbesondere die Bibliotheken `sklearn`, `ipyannotations` und `superintendent` von Bedeutung. Wir nutzen `sklearn`, um Daten von OpenML mit `fetch_openml` zu laden und sie vorab mit `zoom` aus `scipy.ndimage` neu zu skalieren.

Die `ClassLabeller`-Klasse aus `ipyannotations.generic` stellt uns ein grundlegendes Widget zur Verfügung, mit dem wir Daten labeln können. Durch die Integration von `Superintendent` wird die Funktionalität dieses Widgets erweitert, um Active Learning verwenden zu können. Wir importieren auch das Modul `pickle`, um später unser Modell exportieren zu können. Um die Benutzerfreundlichkeit im Widget sicherzustellen, werden Warnungen mit `filterwarnings` unterdrückt.

In [1]:
from ipyannotations.generic import ClassLabeller
import matplotlib.pyplot as plt
import numpy as np
import pickle
from scipy.ndimage import zoom
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from superintendent import Superintendent
from warnings import filterwarnings
filterwarnings("ignore")

### 2. Laden der Daten <a id='Laden-der-Daten'></a>
<!--
Die Daten beziehen wir mit Hilfe der `sklearn.datasets.fetch_openml` Funktion von [OpenML](https://www.openml.org/search?type=data&status=active&id=554).
Da wir ein recht simples Modell verwenden werden, skalieren wir die Daten von 28x28 auf 14x14 Pixel.
Zudem beschränken wir die Menge der Daten auf insgesamt 200 Bilder der Ziffern 0, 1, 2 und 3.
-->

Wir greifen auf die Daten mithilfe der `sklearn.datasets.fetch_openml`-Funktion von [OpenML](https://www.openml.org/search?type=data&status=active&id=554) zu. Da wir ein recht simples Modell verwenden, skalieren wir die Daten von 28x28 auf 14x14 Pixel herunter. Darüber hinaus beschränken wir die Datenauswahl auf insgesamt 200 Bilder, die die Ziffern 0, 1, 2 und 3 repräsentieren.

In [2]:
X, y_true = fetch_openml('mnist_784', parser='auto', return_X_y=True, as_frame=False) #Download MNIST von OpenML
X = zoom(X.reshape(-1,28,28), (1,0.5,0.5)) #Reskaliert die Bilder von 28x28 zu 14x14
idx = np.where(y_true.astype(int) <= 3) # Bestimmt die Indizes für Bilder mit dem Label 0, 1, 2 oder 3
X, y_true = X[idx], y_true[idx] # Selektiert die Daten auf Instanzen der Ziffern 0, 1, 2 und 3
X, y_true = X[:200], y_true[:200] # Reduziert die Menge der Daten auf 200 Datenpunkte
print(f"{X.shape=}\t{y_true.shape=}")

X.shape=(200, 14, 14)	y_true.shape=(200,)


### 3. Initialisiere Widget <a id='Initialisiere-Widget'></a>
Das Widget zum Labeln der Daten wird durch die Bibliothek `ipyannotations` bereitstellt und kann bereits auch als alleinstehendes Widget verwendet werden. `Superintendent` erweitert das Widget später lediglich um ein Modell und eine Sampling Strategie, und integriert so den Active Learning Cycle in das Widget.

Wir definieren vorab eine Methode zur Visualisierung eines einzelnen Datenpunkts und erzeugen damit eine Instanz der Klasse `ipyannotations.generic.ClassLabeller`.

In [3]:
def plot_mnist(x):
    'Plottet ein Bild aud dem Datensatz MNIST'
    plt.figure(figsize=(3,3))
    plt.axis('off')
    plt.imshow(x, cmap="Greys_r")
    plt.show()

# Erzeugt das Widget
annotation_widget = ClassLabeller(
    options=range(4), # Liste alle Klassenlabels
    display_function=plot_mnist,
    allow_freetext=False
)

### 4. Integriere Widget in Active Learning Cycle <a id='#Integriere-Widget-in-Active-Learning-Cycle'></a>
Um eine Instanz der Klasse `Superintendent` zu erzeugen benötigen wir:
- die ungelabelten Daten
- ein Model, welches die Kriterien eines sklearn.BaseEstimator erfüllt
- ein Widget (hier empfiehlt es sich `ipyannotations.generic.ClassLabeller` zu verwenden)
- eine Funktion zur Bestimmung der Reihenfolge in welcher die Daten gelabelt werden sollen (alternativ per String eine vorimplementierte Funktion)
- optional eine Methode für die Vorverarbetung der Datenpunkte 

In [4]:
model = SVC(
    kernel="linear",
    probability=True
)

def preprocess_mnist(x, y):
    return x.reshape(-1, 196), y

data_labeller = Superintendent(
    features=X,
    model=model,
    labelling_widget=annotation_widget,
    model_preprocess=preprocess_mnist, 
    acquisition_function='entropy',
)

data_labeller

Superintendent(children=(HBox(children=(HBox(children=(FloatProgress(value=0.0, description='Progress:', max=1…

Man muss zunächst ein paar Instanzen labeln bevor man ein erstes Model trainieren kann. Nach dem ersten Training werden dann die verbleibenden Daten (Bilder) entsprechend dem gewählten Entscheidungskriterium (hier: Entropy) sortiert und zum labeln angezeigt. Man kann per Klick auf `Retrain` jeder Zeit das Model neu trainieren. Wenn man sich selber nicht ganz sicher ist, um welche Ziffer es sich handeln soll kann man auch mit `Skip`  einzelne Beispiele überspringen. Mit `Undo` wird das Label für das letzte gelabelte Bild wieder zurückgesetzt. `Sort options` hat standardmäßig erstmal keine Funktion. <br> Der angezeigte Score wird durch eine 3-fold cross validation auf den bereits gelabelten Daten bestimmt.
Ist man mit dem Ergebnis zufrieden oder man hat alle Daten gelabelt, kann man das trainierte Model direkt weiter verwenden oder mit `pickle` exportieren. Außerdem sind die Labels in dem Attribut `new_labels` gespeichert und können exportiert werden.

In [5]:
# # Speichert das Model als binäre Datei in ein File
with open("emit/mnist_model.pkl", "wb") as f:
    f.write(pickle.dumps(data_labeller.model))

# Exportiert die Labels aus dem Widget
y = np.array([int(i) if isinstance(i, str) else np.nan for i in data_labeller.new_labels])
np.save("emit/mnist_y.npy", y)
y

array([ 0.,  1.,  2.,  1.,  3.,  1., nan, nan, nan, nan, nan,  1.,  1.,
       nan,  3., nan,  3., nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan,  1., nan, nan, nan, nan, nan, nan, nan,  1., nan, nan,  2.,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,  3.,  1.,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,  1.,  3.,
       nan,  2., nan, nan, nan,  3., nan,  2., nan, nan, nan, nan, nan,
        3., nan, nan,  1., nan,  2., nan, nan, nan, nan, nan, nan,  3.,
       nan, nan,  3., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan,  0., nan, nan, nan, nan,  0., nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,  3., nan,
       nan, nan, nan, nan, nan, nan, nan,  1.,  1., nan, nan, nan, nan,
       nan, nan, nan, nan, nan,  2.,  3., nan, nan, nan, nan, nan, nan,
        1., nan,  2.,  3., nan, nan, nan, nan, nan,  1.,  2.,  2., nan,
        3., nan, nan, nan, nan, nan, nan, nan, nan, nan,  1., na